# Basics of Mobile Robotics - Final Project

**Students: Cécile Chavane, Yucef Grebici, Camille Guillaume, Théo Hermann**

<h1>Table of Contents<span class="tocSkip"></span></h1>

<div class="toc"><ul class="toc-item">
    <li><span><a href="#Introduction" data-toc-modified-id=">Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item">
        <li><span><a href="#Project-description" data-toc-modified-id="Project-description-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Project description</a></span></li>
        <li><span><a href="#Environment-Setup" data-toc-modified-id="Environment-Setup-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Environment Setup</a></span></li>
        <li><span><a href="#Libraries-Import" data-toc-modified-id="Libraires-Import-1.2"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Libraries Import</a></span></li>
        <li><span><a href="#Thymio COnnection" data-toc-modified-id="Thymio-Connection-1.3"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Thymio Connection</a></span></li>
        </ul></li>
    <li><span><a href="#Description-of-the-modules" data-toc-modified-id="Description-of-the-modules-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Description of the Modules</a></span><ul class="toc-item">
        <li><span><a href="#Vision" data-toc-modified-id="Vision-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Vision</a></span></li>
        <li><span><a href="#Global-Navigation" data-toc-modified-id="Global-Navigation-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Global Navigation</a></span></li>
        <li><span><a href="#Local-Navigation" data-toc-modified-id="Local-Navigation-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Local Navigation</a></span></li>
        <li><span><a href="#Filtering" data-toc-modified-id="Filtering-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Filtering</a></span></li>
        <li><span><a href="#Motion-Control" data-toc-modified-id="Motion-Control-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Motion Control</a></span></li>
        </ul></li>
    <li><span><a href="#Code-of-the-overall-project" data-toc-modified-id="Code-of-the-overall-project-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Code of the overall project</a></span></li>
    <li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Conclusion</a></span>
    </li></ul>
    
</div>

# Introduction

## Project description 


**Guidelines:**


**Implementation:**

## Environment  Setup

Description of the environment , obstacles,corners, start/goal, wireless thymio, camera position

<figure>
  <img src="pics/environment.jpeg" width="500" alt="Environment" />
  <figcaption> <center> <u>Figure 1:</u> Picture of the environment setup with lightning <center> </figcaption>
</figure>



<!-- Commentaire : <img src="pics/environment.jpeg" width="500"/> -->

## Libraries import

In [ ]:
#checkez ceux qui sont réellement nécéssaires
import numpy as np
import pyvisgraph as vg
import math
import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Polygon, Point, LineString
import time

## Thymio Connection

In [ ]:
#communication asynchrone avec thymio
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

# Description of the modules

We separated our code in different python file containing the functions for each module. We will import all the python files below

In [ ]:
import Vision as vis
import Global as glob
import local_nav as loc
#import Filtering as fil
import control as ctrl

<span style="color: darkblue"> TO REMOVE 
What is important is not to simply describe the code, which should be readable, but describe what is not in the code: the theory behind, the choices made, the measurements, the choice of parameters etc. Do not forget to cite your sources ! You can of course show how certain modules work in simulation here, or with pre-recorded data.</span> 

## Vision

## Global Navigation

During the initialization of the global path given to the robot we call the high level global_pathplanning function that implements the visibility graph algorithm. Here after are the different steps of this function.
We use two libraries for this purpose:

pyvisgraph for the visibility Graph algorithm: https://github.com/TaipanRex/pyvisgraph
- Algorithm: Given a set of simple obstacle polygons, build a visibility graph and find the shortest path between two points.
- The visibility graph algorithm (D.T. Lee) runs in O(n^2 log n) time. 
- The shortest path is found using Djikstra's algorithm.

geopanda to manage geometric data: https://geopandas.org/en/stable/docs/reference/geoseries.html
- Geometric library that allows to use data of geometric type: Geoseries, polygons, points and to manage this geometric data: plotting, adding marging, input to the visibility graph functions

Here we will define some example polygons that represent the obstacles, and example start and goal positions

In [ ]:
# Initialization
p1 = [(1, 0), (2, 0), (2, 1.5), (1, 1.5)]
p3 = [(3, 3), (4, 3), (4, 4), (3, 4)]
list_obstacles=[p1,p3]
start_point=[0.0,0.0]
end_point=[4.5, 3.5]

In [ ]:
# We convert this polygone list into a geometric set of polygons
g_without_margin = glob.obstacles_to_polygons(list_obstacles)
glob.plot_geometric_data(g_without_margin)

# We add a margin to the obstacles related to the thymio's size, here 0.2 as example
margin=0.2 
g = glob.polygons_add_margin(g_without_margin,margin)
glob.plot_geometric_data(g)

We apply the visibility graph algorithm to this geometric set of Polygons representing the obstacles and compute the shortest path from start to goal



In [ ]:
# Visibility graph creation based on the corners of obstacles with margin
visgraph = glob.polygons_to_VisibilityGraph(g)

# Computation of the shortest path of the visibility graph with Djikstra algorithm
shortest_path = glob.VisibilityGraph_shortest_path(visgraph, start_point, end_point)
print("Shortest path: ", shortest_path)
distance = glob.path_distance(shortest_path)

We convert this shortest path between start and goal into the geometric type and plot the resulting global path 

In [ ]:
path = glob.ShortestPath_to_geometric(shortest_path)

# We plot the geometric data set with both the path and the obstacles
g = g.geometry.append(g_without_margin)
g = g.geometry.append(path.geometry)
glob.plot_geometric_data(g)

Finally we convert this geometric path into a vector which will be given to the main and will correspond to the path steps that the robot will follow if there is no local avoidance.

In [ ]:
path=glob.geometric_path_to_vector(path)

## Local Navigation

## Filtering

## Motion Control

Thymio is a differential drive robot meaning that we can actuate each wheel separately. If we set the same speed for both wheels it results in a straight movement and setting opposite speed produce a rotation on itself. Furthermore each wheel motor command is a combination of the desired forward speed and rotation speed. 

We decided to use the Astolfi controller presented in class to control the motion of the robot, we had to tune three parameters: Kp which is the proportionnal controller for the distance to the next goal

Method used to tune the controller

threshold and fsm

# Code of the overall Project

# Conclusion